In [1]:
import glob #to read the files
import json #to work if .json
import numpy as np #to math
import pandas as pd #to save the data
import math #to convertion calculus
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
from scipy import interpolate #to interpolate the wavelength and flux
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
#import scipy.optimize as opt
from scipy import optimize
from scipy import signal
import os.path
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy import stats
import os.path 

In [4]:
def radec_to_xyz(ra, dec):
    x = math.cos(np.deg2rad(dec)) * math.cos(np.deg2rad(ra))
    y = math.cos(np.deg2rad(dec)) * math.sin(np.deg2rad(ra))
    z = math.sin(np.deg2rad(dec))

    return np.array([x, y, z], dtype=np.float64)

#functions from sncosmo
def cmb_dz(ra, dec):
    """See http://arxiv.org/pdf/astro-ph/9609034
     CMBcoordsRA = 167.98750000 # J2000 Lineweaver
     CMBcoordsDEC = -7.22000000
    """

    # J2000 coords from NED
    CMB_DZ = 371000. / 299792458.
    CMB_RA = 168.01190437
    CMB_DEC = -6.98296811
    CMB_XYZ = radec_to_xyz(CMB_RA, CMB_DEC)

    coords_xyz = radec_to_xyz(ra, dec)
    
    dz = CMB_DZ * np.dot(CMB_XYZ, coords_xyz)

    return dz

def cmb_to_helio(z, ra, dec):
    """Convert from CMB-frame redshift to heliocentric redshift.
    
    Parameters
    ----------
    z : float
        CMB-frame redshift.
    ra, dec: float
        RA and Declination in degrees (J2000).
    """

    dz = -cmb_dz(ra, dec)
    one_plus_z_pec = math.sqrt((1. + dz) / (1. - dz))
    one_plus_z_helio = (1. + z) * one_plus_z_pec

    return one_plus_z_helio - 1.

def one_lenght_redshift(data):
    
    redshift_value = None
    
    #if there is kind in the list continue
    if "kind" in data["redshift"][0]:

        #if its heliocentric pick then
        if data["redshift"][0]["kind"] == "heliocentric":

            redshift_value = float(data["redshift"][0]["value"])
            print(redshift_value)
        
        #if its cmb pick then    
        if data["redshift"][0]["kind"] == "cmb":

            #convertion of hours to degrees
            if data["ra"][0]["u_value"] == "hours" and data["dec"][0]["u_value"] == "degrees":

                c = SkyCoord(str(data["ra"][0]["value"]),str(data["dec"][0]["value"]), unit=(units.hourangle, units.deg))

            elif data["ra"][0]["u_value"] == "hours" and data["dec"][0]["u_value"] == "hours":

                c = SkyCoord(str(data["ra"][0]["value"]),str(data["dec"][0]["value"]), unit=(units.hourangle, units.hourangle))

            else:

                print("Erro!")

            redshift_value = float(cmb_to_helio(float(data["redshift"][0]["value"]), c.ra.deg, c.dec.deg))
            print(redshift_value)
      
    #if there is only 1 and its not specificated
    else:
            
        redshift_value = float(data["redshift"][0]["value"])
        print(redshift_value)
    
    return redshift_value

def redshift_selection(data):
    
    redshift_value = None
    
    for nredshift in range(0,len(data["redshift"])):
            
        #if there is kind in the list continue
        if "kind" in data["redshift"][nredshift]:

            #if its heliocentric pick then
            if data["redshift"][nredshift]["kind"] == "heliocentric":

                redshift_value = float(data["redshift"][nredshift]["value"])
                print(redshift_value)
                break
                
            
            #if its cmb 
            if data["redshift"][nredshift]["kind"] == "cmb":

                #convertion of hours to degrees
                if data["ra"][nredshift]["u_value"] == "hours" and data["dec"][nredshift]["u_value"] == "degrees":

                    c = SkyCoord(str(data["ra"][nredshift]["value"]),str(data["dec"][nredshift]["value"]), unit=(units.hourangle, units.deg))

                elif data["ra"][nredshift]["u_value"] == "hours" and data["dec"][nredshift]["u_value"] == "hours":

                    c = SkyCoord(str(data["ra"][nredshift]["value"]),str(data["dec"][nredshift]["value"]), unit=(units.hourangle, units.hourangle))

                else:

                    print("Erro 1!")

                #convertion of cmb to heliocentric
                redshift_value = float(cmb_to_helio(float(data["redshift"][nredshift]["value"]), c.ra.deg, c.dec.deg))
                print(redshift_value)
                break
            
    return redshift_value


def redshift_HB(data):
    redshift_value = None
    
    maxredshift = None
    
    sumredshift = 0
    HB_test = 0
    for nredshift in range(0,len(data["redshift"])):
            
        #if not realize a sum to obtain a mean value
        if "kind" not in data["redshift"][nredshift]:
            
            sumredshift = sumredshift + float(data["redshift"][nredshift]["value"])
                
            #if the sn is in the hubnle flow realize a mean value of the redshifts
            if float(data["redshift"][nredshift]["value"]) > 0.15:
                
                HB_test = 1
                
            #if not the first redshift is the max redshift
            if nredshift == 0:
                maxredshift = float(data["redshift"][nredshift]["value"])
                
            if nredshift > 0 and maxredshift is not None:
                #then analise if the next redshift is grater than max redshift 
                if float(data["redshift"][nredshift]["value"]) > maxredshift:
                        
                    maxredshift = float(data["redshift"][nredshift]["value"])
        
    if HB_test == 1:
            
        #if sne is in the hubble flow realize a mean value
        redshift_value = sumredshift/len(data["redshift"])
        print(redshift_value)  
    elif HB_test == 0:
            
        #if not the maximum redshift is the heliocentric redshift
        redshift_value = maxredshift
        print(redshift_value)
        
    return redshift_value

def redshift_value(data):
    redshift_func = None
    #if there is 1 value use it
    if len(data["redshift"]) == 1:
        
        redshift_func = one_lenght_redshift(data)

    #if not...
    if len(data["redshift"]) == 1 and redshift_func is None:
        
        print("redshift nao encontrado")
        
    if len(data["redshift"]) > 1:
        
        redshift_func = redshift_selection(data)
                
    if len(data["redshift"]) > 1 and redshift_func is None:
        redshift_func = redshift_HB(data)
        
    return redshift_func

def max_time(data):
   
    t0dat = data["maxdate"][0]["value"]
    t0_max_photo = t0dat.split("/")
    t0_max_photo=str(t0_max_photo[0])+'-'+str(t0_max_photo[1])+'-'+str(t0_max_photo[2])
    t0_max_photo=Time(t0_max_photo).mjd
        
    return t0_max_photo
    
def suavization(x):
    
    #b, a = signal.butter(8, 0.14)
    #y = signal.filtfilt(b, a, x, padlen=0)
    
    return x

In [5]:
sn_names=glob.glob("*.json")
#sn_names=["*.json"]
'''
file = open("newtraining.txt")
lines = file.readlines()
file.close()

sn_names = []

for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()
    sn_names.append(co[0])
'''

'\nfile = open("newtraining.txt")\nlines = file.readlines()\nfile.close()\n\nsn_names = []\n\nfor line in lines:\n    if line.startswith(\'#\'): continue\n    co=line.rstrip().replace(\'INDEF\',\'Nan\').split()\n    sn_names.append(co[0])\n'

In [6]:
sn_names = ["SN2012fr.json"]

In [7]:
#save_path = '/home/user/Área de Trabalho/salt2templates/data'

In [8]:
saltname="salt2_template_0.dat"

In [9]:
file = open(saltname)
lines = file.readlines()
file.close()

In [10]:
x_salt = []
y_salt = []
z_salt = []

#to separate the collumns
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()
        
    #saving in temporary lists
    x_salt.append(co[0])
    y_salt.append(co[1])
    z_salt.append(co[2])

#to float
x_salt = np.array(x_salt, dtype=float)
y_salt = np.array(y_salt, dtype=float)
z_salt = np.array(z_salt, dtype=float)

In [11]:
maxv = []
for i in range(0, 71):
    
    maxv.append(z_salt[721*i + 500])

In [12]:
final_gradex = np.linspace(-20, 50, 71)
final_gradey = np.linspace(2000, 9200, 721)
units = []

list_data = [[]for y in range(0,len(final_gradey))]
number_data_density = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 

for u in range(0,len(sn_names)):
    
    wave = [[] for x in range(0,len(final_gradex))]
    flux = [[] for x in range(0,len(final_gradex))]
    
    print(sn_names[u])

    #reading the .json file
    with open(sn_names[u], "r") as read_file:
        data = json.load(read_file)

    #catching the name of supernova
    url=sn_names[u]
    if url.endswith('.json'):
        url = url[:-5]
    #print(url)
    #defining the lists

    redshift = redshift_value(url)
        
    #print("valor final")    
    #print(redshift)
    
    
    spec_functions = []
    max_min = []
    
    t0 = max_time(url)
    if redshift is not None and t0 is not None:
        
        #spectra data
        i=0
        n=0
        while i < len(data[url]["spectra"]):


            if "time" in data[url]["spectra"][i]:

                spectratime = float(data[url]["spectra"][i]["time"]) - float(t0)
                #print(spectratime)

            alert3 = 0
            #if its the first spectra then previous time is the first time
            if i == 0:
                previoustime = spectratime

            else:
                #if the diference between the time and previous time is more than * days so emmit an alert to continue
                if (spectratime - previoustime) < 2000:

                    alert3 = 1
                    previoustime = spectratime

            #if the diference of time is less than * continue
            if alert3 == 1:

                #verifing if its calibrated and if it have a redshift correction 
                alert1 = 0 #for calibration
                alert2 = 0 #for redshift correction

                if data[url]["spectra"][i]["u_fluxes"] == "erg/s/cm^2/Angstrom":          
                    alert1 = 1 #first alert

                    if "deredshifted" in data[url]["spectra"][i]:
                        alert2 = 1 #second alert

                #if its calibrated and corrected
                if alert1 == 1 and alert2 == 1:

                    #there are 3 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        n = n + 1

                    #there are 2 collumns: wavelength and flux
                    elif len(data[url]["spectra"][i]["data"][0]) == 2:

                        n = n + 1

                #if its calibrated, redshift not adjusted but with a clear redshift
                elif alert1 ==1 and alert2 == 0:   

                    #there are 3 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        n = n + 1

                    #there are 2 collumns: wavelength and flux
                    elif len(data[url]["spectra"][i]["data"][0]) == 2:

                        n = n + 1

            i = i + 1

       
        
        if n > 20:
        

            #print(redshift)
            spectra_data = 0
            i=0
            n=0
            times = []
            max_fluxes = []
            time_verification = 0
            wavelength_verification = 0
            while i < len(data[url]["spectra"]):

                if "time" in data[url]["spectra"][i]:

                    spectratime = float(data[url]["spectra"][i]["time"]) - float(t0)
                    #print(spectratime)
                    #verifing if its calibrated and if it have a redshift correction 
                    alert1 = 0 #for calibration
                    alert2 = 0 #for redshift correction

                    if data[url]["spectra"][i]["u_fluxes"] == "erg/s/cm^2/Angstrom":          
                        alert1 = 1 #first alert

                        if "deredshifted" in data[url]["spectra"][i]:
                            alert2 = 1 #second alert

                    #if its calibrated and corrected
                    if alert1 == 1 and alert2 == 1:
                        #print("ok")
                        #there are 3 collumns: wavelength flux and flux error
                        if len(data[url]["spectra"][i]["data"][0]) == 3:

                            df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux', 'fluxerror'])

                            #converting to float
                            df['wavelength'] = df['wavelength'].astype(float)
                            df['flux'] = df['flux'].astype(float)
                            
                            
                            #sort the data by wavelength
                            df = df.sort_values(by=['wavelength'])
                            df = df.reset_index(drop=True)

                            #to kill negative fluxes
                            fluxtemp = []
                            for var in range(0,len(df['flux'])):
                                if (df['flux'][var] > 0):
                                    fluxtemp.append(df['flux'][var])
                                else:
                                    fluxtemp.append(0)            
                 
                            #making a list of lists 
                            
                            suavf = suavization(fluxtemp)
                                               
                            for x in range(0,len(final_gradex)):
                                
                                if len(wave[x]) ==0:

                                    if final_gradex[x] - 0.5 < spectratime < final_gradex[x] + 0.5:

                                        wave[x].append(df['wavelength'])
                                        flux[x].append(suavf)  

                                
                            units.append(data[url]["spectra"][i]["u_fluxes"])
                        
                        #there are 2 collumns: wavelength and flux
                        if len(data[url]["spectra"][i]["data"][0]) == 2:

                            df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux'])

                            #converting to float
                            df['wavelength'] = df['wavelength'].astype(float)
                            df['flux'] = df['flux'].astype(float)
                            
                            
                            #sort the data by wavelength
                            df = df.sort_values(by=['wavelength'])
                            df = df.reset_index(drop=True)

                            #to kill negative fluxes
                            fluxtemp = []
                            for var in range(0,len(df['flux'])):
                                if (df['flux'][var] > 0):
                                    fluxtemp.append(df['flux'][var])
                                else:
                                    fluxtemp.append(0)            
              
                            #making a list of lists 
                            suavf = suavization(fluxtemp)

                            for x in range(0,len(final_gradex)):
                                
                                if len(wave[x]) ==0:

                                    if final_gradex[x] - 0.5 < spectratime < final_gradex[x] + 0.5:

                                        wave[x].append(df['wavelength'])
                                        flux[x].append(suavf)  
                      
                            units.append(data[url]["spectra"][i]["u_fluxes"])

                            
                    #if its calibrated, redshift not adjusted but with a clear redshift
                    if alert1 ==1 and alert2 == 0:   

                        #there are 3 collumns: wavelength flux and flux error 
                        if len(data[url]["spectra"][i]["data"][0]) == 3:

                            df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux', 'fluxerror'])

                            #converting to float
                            df['wavelength'] = df['wavelength'].astype(float)
                            df['flux'] = df['flux'].astype(float)
                            
                            
                            #sort the data by wavelength
                            df = df.sort_values(by=['wavelength'])
                            df = df.reset_index(drop=True)

                            #to kill negative fluxes
                            fluxtemp = []
                            for var in range(0,len(df['flux'])):
                                if (df['flux'][var] > 0):
                                    fluxtemp.append(df['flux'][var])
                                else:
                                    fluxtemp.append(0)


                            #redshift correction
                            fluxtemp = [x * (((1+redshift))**3) for x in fluxtemp] #for the flux           
                            df['wavelength'] = [x * (1/(1+redshift)) for x in df['wavelength']] #for the wavelength
                            spectratime = spectratime/(1+redshift) #for time
                 
                            #making a list of lists 
                            suavf = suavization(fluxtemp)
                            
                            for x in range(0,len(final_gradex)):
                                
                                if len(wave[x]) ==0:

                                    if final_gradex[x] - 0.5 < spectratime < final_gradex[x] + 0.5:

                                        wave[x].append(df['wavelength'])
                                        flux[x].append(suavf)  

                             
                            units.append(data[url]["spectra"][i]["u_fluxes"])
                            
                            
                        #there are 2 collumns: wavelength flux and flux error 
                        if len(data[url]["spectra"][i]["data"][0]) == 2:

                            df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux'])

                            #converting to float
                            df['wavelength'] = df['wavelength'].astype(float)
                            df['flux'] = df['flux'].astype(float)
                            
                            
                            #sort the data by wavelength
                            df = df.sort_values(by=['wavelength'])
                            df = df.reset_index(drop=True)

                            #to kill negative fluxes
                            fluxtemp = []
                            for var in range(0,len(df['flux'])):
                                if (df['flux'][var] > 0):
                                    fluxtemp.append(df['flux'][var])
                                else:
                                    fluxtemp.append(0)

                            #redshift correction
                            fluxtemp = [x * (((1+redshift))**3) for x in fluxtemp] #for the flux           
                            df['wavelength'] = [x * (1/(1+redshift)) for x in df['wavelength']] #for the wavelength
                            spectratime = spectratime/(1+redshift) #for time
               
                            #making a list of lists 
                            suavf = suavization(fluxtemp)                       
                                                       
                            for x in range(0,len(final_gradex)):
                                
                                if len(wave[x]) ==0:

                                    if final_gradex[x] - 0.5 < spectratime < final_gradex[x] + 0.5:

                                        wave[x].append(df['wavelength'])
                                        flux[x].append(suavf)  
   
                            units.append(data[url]["spectra"][i]["u_fluxes"])
                i = i + 1

SN2012fr.json


TypeError: string indices must be integers

In [ ]:
units

In [ ]:
list_data_temp = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))]

In [ ]:
 for i in range(0,len(list_data)):
    
    if len(list_data[i]) > 0:
        
        for j in range(0,len(list_data[i])):
            
            x = list_data[i][j][0][0]
            y = list_data[i][j][0][1]
            
            for k in range(0,len(x)):
                
                posx = [ii for ii,b in enumerate(final_gradex) if b == x[k]]
                        
                list_data_temp[i][posx[0]].append(y[k])

In [ ]:
mean = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 
std = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 
N = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 

for i in range(0,len(list_data_temp)):
    for j in range(0,len(list_data_temp[i])):
        #print(list_data_temp[i][j])
        
        
        
        if len(list_data_temp[i][j])>0:
            
            for k in range(0,len(list_data_temp[i][j])):
                N[i][j].append(1)
            
            mean[i][j].append(np.mean(list_data_temp[i][j]))
            std[i][j].append(np.std(list_data_temp[i][j]))
            
    

In [ ]:
len(mean)

In [ ]:
mean_s = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 
std_s = [[[] for x in range(0,len(final_gradex))] for y in range(0,len(final_gradey))] 

In [ ]:
for i in range(0,len(mean)):
    x = []
    zm = []
    sm = []
    for j in range(0,len(mean[i])):
        
        if len(mean[i][j])>0:
            
            x.append(final_gradex[j])
            zm.append(mean[i][j])
            sm.append(std[i][j])
            
    if len(x)>0:
        
        bm, am = signal.butter(8, 0.1)
        ym = signal.filtfilt(bm, am, zm, padlen=0)
        
        bd, ad = signal.butter(8, 0.3)
        yd = signal.filtfilt(bd, ad, sm, padlen=0)
        
    for k in range(0,len(x)):
        
        posx = [ii for ii,b in enumerate(final_gradex) if b == x[k]]
        
        
        mean_s[i][posx[0]].append(ym[k])
        std_s[i][posx[0]].append(yd[k])

In [ ]:
xx_test = []
yy_test = []
mean_flux = []
std_flux = []

for i in range(0,len(mean_s)):
    
    for j in range(0,len(mean_s[i])):
        
        if len(mean_s[i][j]) > 0:
        
            yy_test.append(final_gradey[i])
            xx_test.append(final_gradex[j])
            mean_flux.append(mean_s[i][j][0][0])
            std_flux.append(std_s[i][j][0][0])
            

In [ ]:
mean_flux[0]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xx_test, yy_test, std_flux)
#ax.set_zlim3d(0,0.1e-12)

plt.show()

In [ ]:
x_grid = np.linspace(-10, 50, 61)
y_grid = np.linspace(3500, 9200, 600)
B1, B2 = np.meshgrid(x_grid, y_grid, indexing='xy')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import scipy.interpolate as interp
Z = interp.griddata((xx_test,yy_test),std_flux,(B1,B2),method='cubic',fill_value = 0,rescale=True)
#Z = splinemean(B1, B2)
%matplotlib notebook
fig = plt.figure(figsize=(10,6))
ax = axes3d.Axes3D(fig)
ax.plot_wireframe(B1, B2, Z)
ax.plot_surface(B1, B2, Z,alpha=0.2)
#ax.scatter3D(xx_test,yy_test,mean_flux, c='r')

plt.show()

In [ ]:
density_x = []
density_y = []
density_z = []

for i in range(0,len(final_gradex)):
    for j in range(0,len(final_gradey)):
        
        density_x.append(final_gradex[i])
        density_y.append(final_gradey[j])
        density_z.append(sum(N[j][i]))
        
        

In [ ]:
for i in range(0,len(density_z)):
    
    if density_z[i] != 0:
        
        print(density_z[i])

In [ ]:
len(density_z)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib notebook
z = np.reshape(density_z, (71, 721)).T
plt.imshow(z, extent=(np.amin(density_x), np.amax(density_x), np.amin(density_y), np.amax(density_y)), cmap=cm.viridis, aspect='auto', interpolation = 'bilinear')
plt.colorbar()
plt.clim(0,80)
plt.xlabel('time (days)')
plt.ylabel('wavelength ($\AA$)')
plt.savefig('densityplot.png')
plt.show()

In [ ]:
import os.path 

save_path = '/home/user/Área de Trabalho/mathematica_analisys'

table = []
for k in range(0,len(mean_flux)):
    table.append((xx_test[k], yy_test[k], mean_flux[k]))


name_of_file = 'M0'

completeName = os.path.join(save_path, name_of_file+".dat")      
    
    
f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()
